In [16]:
import wrds
import pandas as pd

In [17]:
db = wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [18]:
# Pull monthly market cap to identify top 100 PERMNOs with data over the period
monthly = db.raw_sql("""
    SELECT permno, date, abs(prc)*shrout/1000 as mktcap
    FROM crsp.msf
    WHERE date BETWEEN '2004-01-01' AND '2024-01-01'
""")


In [19]:
# Count months of data per PERMNO
coverage = (
    monthly.groupby('permno')['date']
    .count()
    .reset_index(name='n_months')
)

# Full coverage = 12 months × 20 years = 240 months
full_data = coverage[coverage['n_months'] >= 240]

# Join back to get avg market cap and pick top 100
avg_mktcap = (
    monthly[monthly['permno'].isin(full_data['permno'])]
    .groupby('permno')['mktcap']
    .mean()
    .reset_index(name='avg_mktcap')
)

top100 = avg_mktcap.sort_values(by='avg_mktcap', ascending=False).head(100)
permnos = top100['permno'].tolist()


In [20]:
permno_str = ','.join(map(str, permnos))

prices_df = db.raw_sql(f"""
    SELECT permno, date, shrout, prc AS real_price, cfacpr, prc / cfacpr AS prc
    FROM crsp.dsf
    WHERE date BETWEEN '2004-01-01' AND '2024-12-31'
      AND permno IN ({permno_str})
""")

In [21]:
prices_df[prices_df['cfacpr'] > 1]

,permno,date,shrout,real_price,cfacpr,prc
10570,10145,2004-01-02,862051.0,33.33,1.047784,31.809991
10571,10145,2004-01-05,862051.0,33.5,1.047784,31.972239
10572,10145,2004-01-06,862051.0,33.93,1.047784,32.382628
10573,10145,2004-01-07,862051.0,33.73,1.047784,32.191749
10574,10145,2004-01-08,862051.0,35.35,1.047784,33.73787
...,...,...,...,...,...,...
23564,92655,2005-05-23,641873.0,97.41,2.0,48.705
23565,92655,2005-05-24,641873.0,97.31,2.0,48.655
23566,92655,2005-05-25,641873.0,96.68,2.0,48.34
23567,92655,2005-05-26,641873.0,97.14,2.0,48.57


In [22]:
fundamentals = db.raw_sql("""
    SELECT gvkey, datadate, fqtr, fyr, tic, conm,
           atq, dlcq, dlttq
    FROM comp.fundq
    WHERE indfmt = 'INDL'
      AND datafmt = 'STD'
      AND popsrc = 'D'
      AND consol = 'C'
      AND datadate BETWEEN '2004-01-01' AND '2024-12-31'
""")

print(fundamentals)

         gvkey    datadate  fqtr  fyr     tic                          conm  \
0       001013  2004-01-31     1   10  ADCT.1    ADC TELECOMMUNICATIONS INC   
1       001082  2004-01-31     3    4  SERV.1                 SERVIDYNE INC   
2       001173  2004-01-31     4    1   AIM.1                AEROSONIC CORP   
3       001183  2004-01-31     4    1   IDAI.                      IDNA INC   
4       001240  2004-01-31     4    1   ABS.1               ALBERTSON'S INC   
...        ...         ...   ...  ...     ...                           ...   
467232  353444  2024-12-31     4   12     HLN                    HALEON PLC   
467233  353945  2024-12-31     4   12   ACLLY      ACCELLERON INDUSTRIES AG   
467234  354003  2024-12-31     4   12    BEMB   ISHARES JP MORGAN B U E M B   
467235  356128  2024-12-31     4   12    KSPI  JOINT STOCK COMPANY KASPI KZ   
467236  356289  2024-12-31     2    6    SUUN                SOLARBANK CORP   

              atq      dlcq      dlttq  
0         

In [23]:
ccmlink = db.raw_sql("""
    SELECT gvkey, lpermno as permno, linktype, linkprim,
           linkdt, linkenddt
    FROM crsp.ccmxpf_linktable
    WHERE linktype IN ('LU', 'LC') AND linkprim IN ('P', 'C')
""")

fundamentals['datadate'] = pd.to_datetime(fundamentals['datadate'])
ccmlink['linkdt'] = pd.to_datetime(ccmlink['linkdt'])
ccmlink['linkenddt'] = pd.to_datetime(ccmlink['linkenddt'])

fundamentals_merged = fundamentals.merge(ccmlink, on='gvkey', how='left')

# Filter only rows where Compustat date falls within the valid link period
fundamentals_merged = fundamentals_merged[
    (fundamentals_merged['datadate'] >= fundamentals_merged['linkdt']) &
    ((fundamentals_merged['datadate'] <= fundamentals_merged['linkenddt']) | fundamentals_merged['linkenddt'].isna())
]

In [24]:
prices_df['date'] = pd.to_datetime(prices_df['date'])
fundamentals_merged['datadate'] = pd.to_datetime(fundamentals_merged['datadate'])
prices_df['permno'] = prices_df['permno'].astype(int)
fundamentals_merged['permno'] = fundamentals_merged['permno'].astype(int)
prices_df = prices_df.sort_values(['permno', 'date'])
fundamentals_merged = fundamentals_merged.sort_values(['permno', 'datadate'])
merged_list = []

for permno in prices_df['permno'].unique():
    price_sub = prices_df[prices_df['permno'] == permno].copy()
    fin_sub = fundamentals_merged[fundamentals_merged['permno'] == permno].copy()

    if not price_sub.empty and not fin_sub.empty:
        merged = pd.merge_asof(
            price_sub.sort_values('date'),
            fin_sub.sort_values('datadate'),
            left_on='date',
            right_on='datadate',
            direction='backward'
        )
        merged_list.append(merged)

merged_df = pd.concat(merged_list, ignore_index=True)
merged_df.to_csv("raw_data.csv")